In [876]:
# Initialize list sequence to store names of Countries
countries = [
    'Afghanistan','Albania','Algeria','Angola','Antigua and Barbuda',
    'Argentina','Armenia','Aruba','Australia','Austria','Azerbaijan',
    'The Bahamas','Bahrain','Bangladesh','Barbados','Belarus','Belgium', 
    'Belize','Benin','Bhutan','Bolivia','Bosnia and Herzegovina',
    'Botswana','Brazil','Brunei','Bulgaria','Burkina Faso','Burma',
    'Burundi','Cabo Verde','Cambodia','Cameroon','Canada','Central African Republic',
    'Chad','Chile','China','Colombia','Comoros','Congo, Democratic Republic of the',
    'Congo, Republic of the','Costa Rica',"Cote d'Ivoire",'Croatia','Cuba','Curaçao',
    'Cyprus','Czech Republic', 'Denmark','Djibouti', 'Dominican Republic','Ecuador',
    'Egypt','El Salvador','Equatorial Guinea', 'Eritrea','Estonia','Eswatini','Ethiopia',
    'Fiji', 'Finland', 'France','Gabon','The Gambia','Georgia','Germany','Ghana','Greece',
    'Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Hong Kong','Hungary',
    'Iceland','India','Indonesia','Iran','Iraq','Ireland','Israel','Italy','Jamaica','Japan',
    'Jordan','Kazakhstan','Kenya',"Korea, Democratic People's Republic of",'Korea, Republic of',
    'Kosovo','Kuwait','Kyrgyz Republic',"Laos",'Latvia','Lebanon','Lesotho','Liberia','Lithuania',
    'Luxembourg','Macau','Macedonia','Madagascar','Malawi','Malaysia','Maldives','Mali','Malta',
    'Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia, Federated States of',
    'Moldova','Mongolia','Montenegro','Morocco','Mozambique','Namibia','Nepal','Netherlands',
    'New Zealand','Nicaragua','Niger','Nigeria','Norway','Oman','Pakistan','Palau','Panama',
    'Papua Guinea New','Paraguay','Peru','Philippines','Poland','Portugal','Qatar','Romania',
    'Russia','Rwanda','St. Lucia','St. Vincent and the Grenadines','Saudi Arabia','Senegal','Serbia',
    'Seychelles','Sierra Leone','Singapore','Slovakia','Slovenia','Solomon Islands','South Africa',
    'South Sudan','Spain','Sri Lanka','Sudan','Suriname','Sweden','Switzerland','Syria','Taiwan',
    'Tajikistan','Tanzania','Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia',
    'Turkey','Turkmenistan','Uganda','Ukraine','United Arab Emirates','United Kingdom',
    'United States of America','Uruguay','Uzbekistan','Venezuela', 'Vietnam','Zambia','Zimbabwe',
    'Libya','St. Maarten','Somalia','Yemen' #Libya, St. Marteen, Somalia and Yemen are Special Cases
]


In [100]:
import pycountry
import PyPDF2 
import textract
import re

In [871]:
len(countries)

187

In [995]:
# Function to Parse the pdf file
def parse_file(filename): 
    # Read the file.
    pdfFileObj = open(filename,'rb')
    # The pdfReader variable is the readable object that will be parsed.
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #Parse through all the pages.
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    # The while loop will read each page.
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    # This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
        text = text
    # If the above returns as False, run the OCR library textract to #convert scanned/image based PDF files into text.
    else:
        text = textract.process(fileurl, method='tesseract', language='eng')
    return text

text = parse_file('2018-TIP-Report.pdf')
    

In [988]:
# Function to Clean the parsed pdf
def clean_text(text):
    text = text.strip().replace("\n", "").replace("˜", "fi").replace("˚", "ft").replace("Sincerely,iiiiii","")\
                .replace("Trafiick","Traffick").replace("™", "'").replace("trafiick","traffick")\
                .replace('ﬁprevalence rateﬂ','"prevalence rate"').replace("˚ight","fight")\
                .replace("/", "").replace("  REPUBLIC OF THE:", " REPUBLIC OF THE:")\
                .replace("DEMOCRATIC PEOPLE'S  REPUBLIC OF", "DEMOCRATIC PEOPLE'S REPUBLIC OF")\
                .replace("PAPUA NEW GUINEA:", "PAPUA GUINEA NEW:").replace("⁄", "").replace("\\","")\
                .replace("ST. VINCENT AND  THE GRENADINES", "ST. VINCENT AND THE GRENADINES")\
                .replace("UNITED STATES  OF AMERICA", "UNITED STATES OF AMERICA")\
                .replace("461SPECIAL CASE: LIBYA", " ").replace("462SPECIAL CASE: LIBYA", " ")\
                .replace("463SPECIAL CASE: ST. MAARTEN", " ").replace("464SPECIAL CASE: SOMALIA", " ")\
                .replace("465SPECIAL CASE: SOMALIA", " ").replace("466SPECIAL CASE: YEMEN", " ")\
                .replace("467SPECIAL CASE: YEMEN", " ").replace("468SPECIAL CASE: YEMEN", " ")
    return text 

cleaned_text = clean_text(text)


In [989]:
# Function to Extract each country data from text
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")        
    new_list = []
    a = text.split(countries_list[0])
    for i in range(len(countries_list)-1):
        a = a[1].split(countries_list[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_list)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries_up,cleaned_text)
len(country_info)


187

In [1206]:
country_dict = {i.upper():[{j:k}] for i,j,k in zip(countries, ["INTRO"]*187, country_info)}
special = [s.upper() for s in ['Libya','St. Maarten','Somalia','Yemen']]
special_country_dict = {s:country_dict[s] for s in special}
len(country_dict.values())


187

In [1207]:
special_country_dict


{'LIBYA': [{'INTRO': "Libya is a Special Case for the third consecutive year. The Libyan Government of National Accord (GNA) struggled to gain institutional capacity and the resources to address trafficking, as the government was focused on consolidating control over its territory and countering violent extremism throughout 2017. The judicial system was not fully functioning during the reporting period, as courts in major cities throughout the country have not been operational since 2014. Violence driven by militias, civil unrest, and increased lawlessness continued to plague Libya throughout the reporting period. Extra-legal armed groups continued to fill a security vacuum across the country; such groups varied widely in their make-up and the extent to which they were under the direction of state authorities. These groups also committed human rights abuses, including unlawful killings. During the year, there were continued reports of the sale and trafficking of African migrants for la

In [1210]:
def removekey(d,delete_items):
    r = dict(d)
    for key in delete_items: 
        del r[key]
    return r

normal_country_dict = removekey(country_dict, special)
len(normal_country_dict.values())

183

In [1211]:
def portion_data(country):
    data = normal_country_dict[country]
    tier = ["TIER 1", "TIER 2", "TIER 3"]
    for i in tier:
        length = len(data[0]["INTRO"].split(i))       
        if "TIER 2 WATCH LIST" in data[0]["INTRO"]:
            i = "TIER 2 WATCH LIST"
            append_data(country,i,data)
        elif length == 2 and "TIER 2 WATCH LIST" not in data[0]["INTRO"]: 
            append_data(country,i,data)            
    return data,country_dict


In [1212]:
def append_data(country,i,data):
    data[0]["INTRO"] = data[0]["INTRO"].split(i)[1]
    data.append({"TIER":i})
    intro = data[0]["INTRO"].split(f"RECOMMENDATIONS FOR {country}")
    data[0]["INTRO"] = intro[0]
    recommend = intro[1].split("PROSECUTION")
    data.append({"RECOMMENDATIONS":recommend[0]})
    prosecute = recommend[1].split("PROTECTION")
    data.append({"PROSECUTIONS":prosecute[0]})
    protect = prosecute[1].split("PREVENTION")
    data.append({"PROTECTION":protect[0]})
    prevent = protect[1].split("TRAFFICKING PROFILE")
    data.append({"PREVENTION":prevent[0]})
    data.append({"TRAFFICKING PROFILE":prevent[1]})
    return data

In [1214]:
portion_data("TOGO")

([{'INTRO': 'The Government of Togo does not fully meet the minimum standards for the elimination of trafficking; however, it is making significant efforts to do so. The government demonstrated significant efforts during the reporting period by identifying more trafficking victimsŠwhich for the first time in at least five years included adults. The government decreased its number of trafficking convictions from the previous reporting period, and suspended public awareness raising activities due to a lack of funding. The government did not report efforts to refer or directly assist trafficking victims. Therefore Togo was downgraded to Tier 2 Watch List.'},
  {'TIER': 'TIER 2 WATCH LIST'},
  {'RECOMMENDATIONS': 'Increase efforts to prosecute, convict, and punish traffickers, especially those involved in the trafficking of adults and forced labor of children using the amended penal code; develop and implement standard operating procedures (SOPs) for law enforcement officers, immigration o

In [285]:
country_short = [i[:6] for i in countries]
gender_class = ["men", "women", "boy", "girl", "child"]
trafficking_type = ["forced", "labor", "sex", "bondage"]

In [1217]:
combinations = [[f,s,t] for f in country_short for s in gender_class for t in trafficking_type]
combinations

[['Afghan', 'men', 'forced'],
 ['Afghan', 'men', 'labor'],
 ['Afghan', 'men', 'sex'],
 ['Afghan', 'men', 'bondage'],
 ['Afghan', 'women', 'forced'],
 ['Afghan', 'women', 'labor'],
 ['Afghan', 'women', 'sex'],
 ['Afghan', 'women', 'bondage'],
 ['Afghan', 'boy', 'forced'],
 ['Afghan', 'boy', 'labor'],
 ['Afghan', 'boy', 'sex'],
 ['Afghan', 'boy', 'bondage'],
 ['Afghan', 'girl', 'forced'],
 ['Afghan', 'girl', 'labor'],
 ['Afghan', 'girl', 'sex'],
 ['Afghan', 'girl', 'bondage'],
 ['Afghan', 'child', 'forced'],
 ['Afghan', 'child', 'labor'],
 ['Afghan', 'child', 'sex'],
 ['Afghan', 'child', 'bondage'],
 ['Albani', 'men', 'forced'],
 ['Albani', 'men', 'labor'],
 ['Albani', 'men', 'sex'],
 ['Albani', 'men', 'bondage'],
 ['Albani', 'women', 'forced'],
 ['Albani', 'women', 'labor'],
 ['Albani', 'women', 'sex'],
 ['Albani', 'women', 'bondage'],
 ['Albani', 'boy', 'forced'],
 ['Albani', 'boy', 'labor'],
 ['Albani', 'boy', 'sex'],
 ['Albani', 'boy', 'bondage'],
 ['Albani', 'girl', 'forced'],
 ['Al

In [ ]:
b = [i for i in text for j in combinations if all(k in i for k in j)]


In [ ]:
b